In [1]:
import argparse
import os
import sys
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image, ImageOps
import pandas as pd
from tqdm import tqdm
import json
import copy
from utils import *

In [2]:
with open('./data/pano_tracks_2.json', 'r') as f:
    pano_tracks = json.load(f)
    
with open('./data/pano_bridges_2.json', 'r') as f:
    pano_bridges = json.load(f)    

In [3]:
n_frames = len(pano_tracks)

In [4]:
tracking_raw = {}
for frame in pano_tracks:
    for instance in pano_tracks[frame]:
        location = instance[:2]
        track_id = instance[2]
        if track_id not in list(tracking_raw.keys()):
            tracking_raw[track_id] = []
            tracking_raw[track_id].append([float(location[0]),float(location[1]),int(frame)])
        else:
            tracking_raw[track_id].append([float(location[0]),float(location[1]),int(frame)])

In [5]:
from scipy.ndimage.filters import gaussian_filter1d

tracking_sm = {}
for k in tracking_raw:
    location = tracking_raw[k]
    if len(location)>5:
        location = np.stack(location)
        location[:,0] = gaussian_filter1d(location[:,0],0.5)
        location[:,1] = gaussian_filter1d(location[:,1],0.5)
        tracking_sm[k] = location
    else:
        tracking_sm[k] = location

In [6]:
tracking = {k:[] for k in range(2,n_frames+2)}

In [7]:
for k in tracking_sm:
    track_id = k
    for t in tracking_sm[k]:
        frame = int(t[2])
        tracking[frame].append([t[0],t[1],track_id])

In [10]:
with open('./data/video_1_pano_smooth_2.json', 'w') as fp:
    json.dump(tracking, fp)
    
with open('./data/video_1_single_smooth_2.json', 'w') as fp:
    json.dump(pano_bridges, fp)